In [21]:
import config
from numpy import ones,vstack
from numpy.linalg import lstsq
import pandas as pd
import numpy as np
np.warnings.filterwarnings('ignore')


In [22]:
years = list(range(config.startingYear, config.finalYear + 1))

In [23]:
def positionedTeams(df, gamesPlayed, teams, count, returnDict):
    team = {"PG": None, "SG": None, "SF": None, "PF": None, "C": None}
    
    for index, row in df.iterrows():
        
        if(team[row.Position.split('-')[0]] == None and row.Games >= gamesPlayed):
            team[row.Position.split('-')[0]] = row.PlayerName
            df.drop(index, inplace=True)

    returnDict[count] = team

    if(count < teams):
        remaining_df = df[df["Games"] >= gamesPlayed]
        positionedTeams(remaining_df, gamesPlayed, teams, count + 1, returnDict)
        
    return returnDict


def guardsAndWings(df, gamesPlayed, teams, count, returnDict):

    guards = [row.PlayerName for index, row in df.iterrows() if (row.Position.split('-')[0] == "PG" or row.Position.split('-')[0] == "SG") and row.Games >= gamesPlayed][:2]
    forwards = [row.PlayerName for index, row in df.iterrows() if (row.Position.split('-')[0] == "SF" or row.Position.split('-')[0] == "PF") and row.Games >= gamesPlayed][:2]
    center = [row.PlayerName for index, row in df.iterrows() if row.Position.split('-')[0] == "C" and row.Games >= gamesPlayed][:1]

    team = {"G": guards, "F": forwards, "C": center[0] if center else None}

    df.drop(df[(df['PlayerName'].isin(guards)) | (df['PlayerName'].isin(forwards)) | (df['PlayerName'].isin(center))].index, inplace=True)

    returnDict.append(team)

    if(count < teams):
        remaining_df = df[df["Games"] >= gamesPlayed]
        guardsAndWings(remaining_df, gamesPlayed, teams, count + 1, returnDict)
    
    return returnDict

def positionlessTeams(df, gamesPlayed, teams, count, returnDict):

    team = [row.PlayerName for index, row in df.iterrows() if row.Games >= gamesPlayed][:5]

    df.drop(df[(df['PlayerName'].isin(team))].index, inplace=True)

    returnDict.append(team)

    if(count < teams):
        remaining_df = df[df["Games"] >= gamesPlayed]
        positionlessTeams(remaining_df, gamesPlayed, teams, count + 1, returnDict)
    
    return returnDict

In [24]:
# Create combined csv with stats over time


for year in years:

    print("-" * 80)
    print("THE YEAR IS " + str(year))
    
    fileName = "finalRatings/fr_{}.csv".format(year)
    df = pd.read_csv(fileName)

    df = df.sort_values('scaledRatingSum', ascending=False)

    # 3 ALL NBA Teams 
    
    allNBATeams = positionedTeams(df.copy(), 0, 3, 1, returnDict={})

    print("All NBA team with positions")

    for index, team in enumerate(allNBATeams):
        print("All NBA " + str(index + 1))
        print(allNBATeams[team])

    # 3 ALL NBA Teams with Games Played Limit
    
    allNBATeams = positionedTeams(df.copy(), 65, 3, 1, returnDict={})

    print("All NBA team with positions and games played limit")

    for index, team in enumerate(allNBATeams):
        print("All NBA " + str(index + 1))
        print(allNBATeams[team])
    
    # 3 ALL NBA Teams with Games Played and Postionless

    allNBATeams = positionlessTeams(df.copy(), 65, 3, 1, returnDict=[])
    
    print("All NBA positionless and games played limit")

    for index, team in enumerate(allNBATeams):
        print("All NBA " + str(index + 1))
        print(team)

    df = df.sort_values('DefensiveRating', ascending=False)

    # 2 All Defensive Teams 

    allDefensiveTeam = guardsAndWings(df.copy(), 0, 2, 1, returnDict=[])

    for index, x in enumerate(allDefensiveTeam):
        print("All DEFENSE " + str(index + 1))
        print(x)

    # 2 All Defensive Teams with Games Played Limit

    allDefensiveTeam = guardsAndWings(df.copy(), 65, 2, 1, returnDict=[])

    for index, x in enumerate(allDefensiveTeam):
        print("All DEFENSE " + str(index + 1))
        print(x)


--------------------------------------------------------------------------------
THE YEAR IS 2023
All NBA team with positions
All NBA 1
{'PG': 'Luka Dončić', 'SG': 'Donovan Mitchell', 'SF': 'Jimmy Butler', 'PF': 'Giannis Antetokounmpo', 'C': 'Nikola Jokić'}
All NBA 2
{'PG': 'Shai Gilgeous-Alexander', 'SG': 'Anthony Edwards', 'SF': 'Kawhi Leonard', 'PF': 'Jayson Tatum', 'C': 'Joel Embiid'}
All NBA 3
{'PG': 'Tyrese Haliburton', 'SG': 'Paul George', 'SF': 'DeMar DeRozan', 'PF': 'Kevin Durant', 'C': 'Anthony Davis'}
All NBA team with positions and games played limit
All NBA 1
{'PG': 'Shai Gilgeous-Alexander', 'SG': 'Donovan Mitchell', 'SF': 'DeMar DeRozan', 'PF': 'Jayson Tatum', 'C': 'Nikola Jokić'}
All NBA 2
{'PG': 'Jalen Brunson', 'SG': 'Anthony Edwards', 'SF': 'Lauri Markkanen', 'PF': 'Julius Randle', 'C': 'Domantas Sabonis'}
All NBA 3
{'PG': 'Fred VanVleet', 'SG': 'Dejounte Murray', 'SF': 'Jaylen Brown', 'PF': 'Evan Mobley', 'C': 'Kristaps Porziņģis'}
All NBA positionless and games pla